# J值策略的计算

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import shutil
import baostock as bs
import talib
import tushare as ts

## 类函数 策略

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# 加入 sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [4]:
"""
TradingStrategyA: J值转为负值
TradingStrategyB: J值由负值转为正值
TradingStrategyC: J值由负值转为正值，且J值差值大于30
"""
from helper.data_loader import DataLoader
from helper.strategy import TradingStrategyA, TradingStrategyB, TradingStrategyC
from helper.technical_analysis import TechnicalAnalysis

In [9]:
def get_strategy_columns(strategy_name):
    """根据策略类型返回需要显示的列"""
    base_columns = ['code', 'code_name', '信号日期']
    
    if strategy_name == 'A':
        return base_columns + [
            '当日收盘价', '5日均线', '20日均线', '60日均线', 
            '当日J值', '前一日J值',
            '5日收益率', '10日收益率', '30日收益率'
        ]
    elif strategy_name == 'B':
        return base_columns + [
            '当日收盘价', '5日均线', '10日均线', '60日均线', 
            '当日J值', '前一日J值',
            '5日收益率', '10日收益率', '30日收益率'
        ]
    elif strategy_name == 'C':
        return base_columns + [
            'D1收盘价', 'D2收盘价', 'D1-D2收益率',
            'D1_5日均线', 'D1_10日均线', 'D1_60日均线',
            'D1_J值', 'D2_J值', 'J值差值',
            'D1_WR14', 'D2_WR14', 'D1_WR28', 'D2_WR28',
            'D1_MACD_DIF', 'D1_MACD_DEA', 'D1_MACD',
            'D2_MACD_DIF', 'D2_MACD_DEA', 'D2_MACD',
            'D1_BOLL中轨', 'D1_BOLL上轨', 'D1_BOLL下轨',
            'D2_BOLL中轨', 'D2_BOLL上轨', 'D2_BOLL下轨',
            '持仓天数'
        ]

## 获取数据

In [5]:
"""沪深300成分股"""
lg = bs.login()
if lg.error_code != '0':
    print(f'登录失败，错误代码：{lg.error_code}, 错误信息：{lg.error_msg}')
else:
    print('登录成功')

# 获取沪深300成分股
rs = bs.query_hs300_stocks()
if rs.error_code != '0':
    print(f'查询沪深300成分股失败，错误代码：{rs.error_code}, 错误信息：{rs.error_msg}')
else:
    # 保存成DataFrame
    hs300_stocks = []
    while rs.next():
        hs300_stocks.append(rs.get_row_data())
    df_hs300 = pd.DataFrame(hs300_stocks, columns=rs.fields)
    print(df_hs300)

bs.logout()

login success!
登录成功
     updateDate       code code_name
0    2025-04-21  sh.600000      浦发银行
1    2025-04-21  sh.600009      上海机场
2    2025-04-21  sh.600010      包钢股份
3    2025-04-21  sh.600011      华能国际
4    2025-04-21  sh.600015      华夏银行
..          ...        ...       ...
295  2025-04-21  sz.300832       新产业
296  2025-04-21  sz.300896       爱美客
297  2025-04-21  sz.300979      华利集团
298  2025-04-21  sz.300999       金龙鱼
299  2025-04-21  sz.301269      华大九天

[300 rows x 3 columns]
logout success!


In [6]:
df_hs300

,updateDate,code,code_name
0,2025-04-21,sh.600000,浦发银行
1,2025-04-21,sh.600009,上海机场
2,2025-04-21,sh.600010,包钢股份
3,2025-04-21,sh.600011,华能国际
4,2025-04-21,sh.600015,华夏银行
...,...,...,...
295,2025-04-21,sz.300832,新产业
296,2025-04-21,sz.300896,爱美客
297,2025-04-21,sz.300979,华利集团
298,2025-04-21,sz.300999,金龙鱼


In [7]:
"""
25年
"""
lg = bs.login()
if lg.error_code != '0':
    print(f'登录失败，错误代码：{lg.error_code}, 错误信息：{lg.error_msg}')
else:
    print('登录成功')

# 设置日期范围
start_date = '2025-01-01'
end_date = '2025-4-26'

# 初始化存储数据的列表
stock_data_2025 = []

# 遍历每只成分股
for code in df_hs300['code']:
    # 获取日K线数据
    rs = bs.query_history_k_data_plus(
        code,
        "date,code,open,high,low,close,preclose,volume,amount,turn",
        start_date=start_date, end_date=end_date,
        frequency="d", adjustflag="3"
    )
    if rs.error_code != '0':
        print(f'查询股票 {code} 数据失败，错误代码：{rs.error_code}, 错误信息：{rs.error_msg}')
        continue

    # 保存数据
    stock_data = []
    while rs.next():
        stock_data.append(rs.get_row_data())
    df_stock = pd.DataFrame(stock_data, columns=rs.fields)
    stock_data_2025.append(df_stock)

# 合并所有股票的数据
df_stock_2025 = pd.concat(stock_data_2025, ignore_index=True)

# 登出系统
bs.logout()

login success!
登录成功
logout success!


In [8]:
df_stock_2025

,date,code,open,high,low,close,preclose,volume,amount,turn
0,2025-01-02,sh.600000,10.3000,10.4200,10.0500,10.1300,10.2900,78907129,803904040.2300,0.268800
1,2025-01-03,sh.600000,10.1200,10.2200,10.0200,10.0600,10.1300,50693944,511576493.6000,0.172700
2,2025-01-06,sh.600000,10.1300,10.2000,9.8100,10.1500,10.0600,79801276,804593503.5600,0.271900
3,2025-01-07,sh.600000,10.1100,10.3000,10.1100,10.2700,10.1500,40265146,411488491.5100,0.137200
4,2025-01-08,sh.600000,10.2600,10.3700,10.2000,10.3000,10.2700,48237450,496301044.8100,0.164300
...,...,...,...,...,...,...,...,...,...,...
22495,2025-04-21,sz.301269,120.8000,124.6000,119.8100,123.9100,120.9800,4083840,499746858.2000,1.556800
22496,2025-04-22,sz.301269,123.0100,124.5000,121.6100,122.8000,123.9100,3159912,389129941.6000,1.204600
22497,2025-04-23,sz.301269,123.0000,123.4800,120.2000,121.2000,122.8000,3464362,420032149.6200,1.320700
22498,2025-04-24,sz.301269,120.7000,121.1900,118.7200,119.4000,121.2000,2866592,343312144.0000,1.092800


## 计算

In [19]:
"""
策略A
"""
stock_data_path = "/home/kennys/experiment/QuantTrading/dataset/沪深300-2025年至今数据.csv"
hs300_constituents_path="/home/kennys/experiment/QuantTrading/dataset/沪深300成分股.csv"
data_loader = DataLoader(stock_data_path=stock_data_path, hs300_constituents_path=hs300_constituents_path)

stock_data = data_loader.load_stock_data()
hs300_constituents = data_loader.load_hs300_constituents()

stock_data = pd.merge(stock_data, hs300_constituents[['code', 'code_name']], on='code', how='inner')

strategy = TradingStrategyA()
date_col = '信号日期'

prepared_data = strategy.prepare_data(stock_data)
signals = strategy.find_trading_signals(prepared_data, ma_type="ma60")

signals = pd.merge(signals, hs300_constituents[['code', 'code_name']], on='code', how='left')

# 计算不同时间段的收益率
returns_5 = strategy.calculate_returns(prepared_data, signals, days=5)
returns_10 = strategy.calculate_returns(prepared_data, signals, days=10)
returns_30 = strategy.calculate_returns(prepared_data, signals, days=30)

print("\n=== 策略回测结果 ===")
print(f"找到的交易信号总数: {len(signals)}")

if len(signals) == 0:
    print("\n无交易信号")

print("\n=== 交易信号明细 ===")

# 将日期转换为字符串格式
signals[f'{date_col}_str'] = pd.to_datetime(signals[date_col]).dt.strftime('%Y-%m-%d')

# 添加不同时间段的收益率到signals
for days, returns in [(5, returns_5), (10, returns_10), (30, returns_30)]:
    if not returns.empty:
        # 将returns中的日期也转换为字符串格式
        returns['signal_date_str'] = returns['signal_date'].dt.strftime('%Y-%m-%d')
        signals = pd.merge(signals, returns[['code', 'signal_date_str', 'return']], 
                            left_on=['code', f'{date_col}_str'], 
                            right_on=['code', 'signal_date_str'], 
                            how='left')
        signals = signals.rename(columns={'return': f'{days}日收益率'})
        signals = signals.drop(['signal_date_str'], axis=1)
    else:
        signals[f'{days}日收益率'] = None

# 删除临时列
signals = signals.drop([f'{date_col}_str'], axis=1)

# 格式化日期显示
signals[date_col] = pd.to_datetime(signals[date_col]).dt.strftime('%Y-%m-%d')

# 获取需要显示的列
display_columns = [
    'code', 'code_name', '信号日期', 
    '当日收盘价', '5日均线', '20日均线', '60日均线', 
    '当日J值', '前一日J值',
    '5日收益率', '10日收益率', '30日收益率'
]

# 对数值列进行四舍五入
numeric_columns = [col for col in signals.columns if col not in ['code', 'code_name', date_col]]
signals[numeric_columns] = signals[numeric_columns].round(2)

# output_path = f"strategy_{strategy_name}-20250427.csv"
# signals[display_columns].to_csv(output_path, index=False, encoding='utf-8-sig')
# print(f"\n交易信号明细已保存至: {os.path.abspath(output_path)}")

print("\n每个交易信号的详细信息:")
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
print(signals[display_columns].to_string(index=False))

# 打印收益率统计信息
print(f"\n=== 收益率统计 ===")
for days, returns in [(5, returns_5), (10, returns_10), (30, returns_30)]:
    if not returns.empty:
        print(f"\n{days}天平均收益率: {returns['return'].mean():.2f}%")
        print(f"{days}天收益率中位数: {returns['return'].median():.2f}%")
        print(f"{days}天胜率: {(returns['return'] > 0).mean() * 100:.2f}%")
    else:
        print(f"\n{days}天收益率数据不足")
    
print(f"\n=== 信号时间分布 ===")
signals_by_month = signals.groupby(pd.to_datetime(signals[date_col]).dt.to_period('M')).size()
print("\n每月信号数量:")
print(signals_by_month)


=== 策略回测结果 ===
找到的交易信号总数: 26

=== 交易信号明细 ===

每个交易信号的详细信息:
     code code_name       信号日期  当日收盘价   5日均线  20日均线  60日均线   当日J值  前一日J值  5日收益率  10日收益率 30日收益率
sh.601899      紫金矿业 2025-04-03  17.11  17.76  17.54  16.64  -6.28   0.09   0.23    2.98   None
sh.603259      药明康德 2025-04-03  63.52  66.58  65.68  60.34  -5.45  16.21 -17.98  -16.80   None
sh.603799      华友钴业 2025-04-03  33.23  33.96  36.18  32.32  -1.41   1.74  -7.52   -4.06   None
sz.002142      宁波银行 2025-04-03  25.36  25.78  25.59  25.22  -5.72  11.35  -6.62   -3.00   None
sh.600161      天坛生物 2025-04-17  20.82  21.66  21.14  20.51  -3.26  19.22  -3.27     NaN   None
sz.001289      龙源电力 2025-04-18  16.38  16.78  16.70  16.14  -3.46   6.96   8.91     NaN   None
sz.000876       新希望 2025-04-21  10.05  10.06   9.91   9.54  -0.27   8.08    NaN     NaN   None
sh.600111      北方稀土 2025-04-22  23.16  23.72  23.62  23.05  -7.53  18.17    NaN     NaN   None
sh.600406      国电南瑞 2025-04-22  23.06  23.35  22.87  22.98  -5.15  18.51    NaN     N

In [ ]:
"""
策略B
"""
stock_data_path = "/home/kennys/experiment/QuantTrading/dataset/沪深300-2025年至今数据.csv"
hs300_constituents_path="/home/kennys/experiment/QuantTrading/dataset/沪深300成分股.csv"
data_loader = DataLoader(stock_data_path=stock_data_path, hs300_constituents_path=hs300_constituents_path)

stock_data = data_loader.load_stock_data()
hs300_constituents = data_loader.load_hs300_constituents()

stock_data = pd.merge(stock_data, hs300_constituents[['code', 'code_name']], on='code', how='inner')

# 选择策略
strategy = TradingStrategyB()
date_col = 'D1日期'  # 策略B使用'D1日期'作为日期列


prepared_data = strategy.prepare_data(stock_data)
signals = strategy.find_trading_signals(prepared_data, ma_type="ma20")

signals = pd.merge(signals, hs300_constituents[['code', 'code_name']], on='code', how='left')

# 计算不同时间段的收益率
returns_5 = strategy.calculate_returns(prepared_data, signals, days=5)
returns_10 = strategy.calculate_returns(prepared_data, signals, days=10)
returns_30 = strategy.calculate_returns(prepared_data, signals, days=30)

print("\n=== 策略回测结果 ===")
print(f"找到的交易信号总数: {len(signals)}")

if len(signals) == 0:
    print("\n无交易信号")

print("\n=== 交易信号明细 ===")

# 将日期转换为字符串格式
signals[f'{date_col}_str'] = pd.to_datetime(signals[date_col]).dt.strftime('%Y-%m-%d')

# 添加不同时间段的收益率到signals
for days, returns in [(5, returns_5), (10, returns_10), (30, returns_30)]:
    if not returns.empty:
        # 将returns中的日期也转换为字符串格式
        returns['signal_date_str'] = returns['signal_date'].dt.strftime('%Y-%m-%d')
        signals = pd.merge(signals, returns[['code', 'signal_date_str', 'return']], 
                            left_on=['code', f'{date_col}_str'], 
                            right_on=['code', 'signal_date_str'], 
                            how='left')
        signals = signals.rename(columns={'return': f'{days}日收益率'})
        signals = signals.drop(['signal_date_str'], axis=1)
    else:
        signals[f'{days}日收益率'] = None

# 删除临时列
signals = signals.drop([f'{date_col}_str'], axis=1)

# 格式化日期显示
signals[date_col] = pd.to_datetime(signals[date_col]).dt.strftime('%Y-%m-%d')

# 获取需要显示的列
display_columns = [
    'code', 'code_name', 'D1日期', 'D2日期',
    'D1收盘价', 'D2收盘价', 'D1-D2收益率',
    'D1_5日均线', 'D1_20日均线', 'D1_60日均线',
    'D1_J值', 'D2_J值',
    'D1_WR14', 'D2_WR14', 'D1_WR28', 'D2_WR28',
    'D1_MACD_DIF', 'D1_MACD_DEA', 'D1_MACD',
    'D2_MACD_DIF', 'D2_MACD_DEA', 'D2_MACD',
    'D1_BOLL中轨', 'D1_BOLL上轨', 'D1_BOLL下轨',
    'D2_BOLL中轨', 'D2_BOLL上轨', 'D2_BOLL下轨',
    '持仓天数'
]

# 对数值列进行四舍五入
numeric_columns = [col for col in signals.columns if col not in ['code', 'code_name', date_col]]
signals[numeric_columns] = signals[numeric_columns].round(2)

# output_path = f"strategy_{strategy_name}-20250427.csv"
# signals[display_columns].to_csv(output_path, index=False, encoding='utf-8-sig')
# print(f"\n交易信号明细已保存至: {os.path.abspath(output_path)}")

print("\n每个交易信号的详细信息:")
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
print(signals[display_columns].to_string(index=False))

# 打印收益率统计信息
print(f"\n=== 收益率统计 ===")
for days, returns in [(5, returns_5), (10, returns_10), (30, returns_30)]:
    if not returns.empty:
        print(f"\n{days}天平均收益率: {returns['return'].mean():.2f}%")
        print(f"{days}天收益率中位数: {returns['return'].median():.2f}%")
        print(f"{days}天胜率: {(returns['return'] > 0).mean() * 100:.2f}%")
    else:
        print(f"\n{days}天收益率数据不足")
    
print(f"\n=== 信号时间分布 ===")
signals_by_month = signals.groupby(pd.to_datetime(signals[date_col]).dt.to_period('M')).size()
print("\n每月信号数量:")
print(signals_by_month)


=== 策略回测结果 ===
找到的交易信号总数: 35

=== 交易信号明细 ===

每个交易信号的详细信息:
     code code_name       D1日期       D2日期  D1收盘价  D2收盘价  D1-D2收益率  D1_5日均线  D1_20日均线  D1_60日均线  D1_J值  D2_J值  D1_WR14  D2_WR14  D1_WR28  D2_WR28  D1_MACD_DIF  D1_MACD_DEA  D1_MACD  D2_MACD_DIF  D2_MACD_DEA  D2_MACD  D1_BOLL中轨  D1_BOLL上轨  D1_BOLL下轨  D2_BOLL中轨  D2_BOLL上轨  D2_BOLL下轨  持仓天数
sh.600031      三一重工 2025-03-21 2025-03-24  19.59  19.96      1.89    19.96     19.40       NaN -12.22   5.34   -52.02   -50.97   -29.07   -20.52         0.73         0.85    -0.23         0.70         0.82    -0.25      19.40      21.36      17.45      19.51      21.34      17.68     3
sh.600039      四川路桥 2025-03-31 2025-04-01   7.94   8.00      0.76     8.08      7.86       NaN  -2.61   3.86   -55.00   -60.32   -29.33   -27.54         0.22         0.24    -0.04         0.20         0.23    -0.06       7.86       8.51       7.21       7.89       8.50       7.28     1
sh.600050      中国联通 2025-03-03 2025-03-06   6.22   6.53      4.98     6.42     

In [ ]:
"""
策略C
"""
stock_data_path = "/home/kennys/experiment/QuantTrading/dataset/沪深300-2025年至今数据.csv"
hs300_constituents_path="/home/kennys/experiment/QuantTrading/dataset/沪深300成分股.csv"
data_loader = DataLoader(stock_data_path=stock_data_path, hs300_constituents_path=hs300_constituents_path)

stock_data = data_loader.load_stock_data()
hs300_constituents = data_loader.load_hs300_constituents()

stock_data = pd.merge(stock_data, hs300_constituents[['code', 'code_name']], on='code', how='inner')

# 选择策略
strategy = TradingStrategyC(j_diff_threshold=60)
date_col = 'D1日期'

prepared_data = strategy.prepare_data(stock_data)
signals = strategy.find_trading_signals(prepared_data, ma_type="ma20")

signals = pd.merge(signals, hs300_constituents[['code', 'code_name']], on='code', how='left')

# 计算不同时间段的收益率
returns_5 = strategy.calculate_returns(prepared_data, signals, days=5)
returns_10 = strategy.calculate_returns(prepared_data, signals, days=10)
returns_30 = strategy.calculate_returns(prepared_data, signals, days=30)

print("\n=== 策略回测结果 ===")
print(f"找到的交易信号总数: {len(signals)}")

if len(signals) == 0:
    print("\n无交易信号")

print("\n=== 交易信号明细 ===")

# 将日期转换为字符串格式
signals[f'{date_col}_str'] = pd.to_datetime(signals[date_col]).dt.strftime('%Y-%m-%d')

# 添加不同时间段的收益率到signals
for days, returns in [(5, returns_5), (10, returns_10), (30, returns_30)]:
    if not returns.empty:
        # 将returns中的日期也转换为字符串格式
        returns['signal_date_str'] = returns['signal_date'].dt.strftime('%Y-%m-%d')
        signals = pd.merge(signals, returns[['code', 'signal_date_str', 'return']], 
                            left_on=['code', f'{date_col}_str'], 
                            right_on=['code', 'signal_date_str'], 
                            how='left')
        signals = signals.rename(columns={'return': f'{days}日收益率'})
        signals = signals.drop(['signal_date_str'], axis=1)
    else:
        signals[f'{days}日收益率'] = None

# 删除临时列
signals = signals.drop([f'{date_col}_str'], axis=1)

# 格式化日期显示
signals[date_col] = pd.to_datetime(signals[date_col]).dt.strftime('%Y-%m-%d')

# 获取需要显示的列
display_columns = [
    'code', 'code_name', 'D1日期', 'D2日期',
    'D1收盘价', 'D2收盘价', 'D1-D2收益率',
    'D1_5日均线', 'D1_20日均线', 'D1_60日均线',
    'D1_J值', 'D2_J值', 'J值差值',
    'D1_WR14', 'D2_WR14', 'D1_WR28', 'D2_WR28',
    'D1_MACD_DIF', 'D1_MACD_DEA', 'D1_MACD',
    'D2_MACD_DIF', 'D2_MACD_DEA', 'D2_MACD',
    'D1_BOLL中轨', 'D1_BOLL上轨', 'D1_BOLL下轨',
    'D2_BOLL中轨', 'D2_BOLL上轨', 'D2_BOLL下轨',
    '持仓天数'
]

# 对数值列进行四舍五入
numeric_columns = [col for col in signals.columns if col not in ['code', 'code_name', date_col]]
signals[numeric_columns] = signals[numeric_columns].round(2)

# output_path = f"strategy_{strategy_name}-20250427.csv"
# signals[display_columns].to_csv(output_path, index=False, encoding='utf-8-sig')
# print(f"\n交易信号明细已保存至: {os.path.abspath(output_path)}")

print("\n每个交易信号的详细信息:")
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
print(signals[display_columns].to_string(index=False))

# 打印收益率统计信息
print(f"\n=== 收益率统计 ===")
for days, returns in [(5, returns_5), (10, returns_10), (30, returns_30)]:
    if not returns.empty:
        print(f"\n{days}天平均收益率: {returns['return'].mean():.2f}%")
        print(f"{days}天收益率中位数: {returns['return'].median():.2f}%")
        print(f"{days}天胜率: {(returns['return'] > 0).mean() * 100:.2f}%")
    else:
        print(f"\n{days}天收益率数据不足")
    
print(f"\n=== 信号时间分布 ===")
signals_by_month = signals.groupby(pd.to_datetime(signals[date_col]).dt.to_period('M')).size()
print("\n每月信号数量:")
print(signals_by_month)


=== 策略C调试信息 ===
初始信号数量: 42

初始信号示例:
           code       date  close      kdj_j     prev_j
1175  sh.600031 2025-03-21  19.59 -12.224966   6.867436
1331  sh.600039 2025-03-31   7.94  -2.614764  17.699789
1461  sh.600050 2025-03-03   6.22  -9.636816   8.690523
2149  sh.600160 2025-03-20  25.03  -1.122448  31.717819
2199  sh.600161 2025-02-13  20.31 -12.860871   3.441245

股票 sh.600031 的J值变化:
D1日期: 2025-03-21 00:00:00, D1_J值: -12.22
D2日期: 2025-03-24 00:00:00, D2_J值: 5.34
J值差值: 17.57

股票 sh.600039 的J值变化:
D1日期: 2025-03-31 00:00:00, D1_J值: -2.61
D2日期: 2025-04-01 00:00:00, D2_J值: 3.86
J值差值: 6.47

股票 sh.600050 的J值变化:
D1日期: 2025-03-03 00:00:00, D1_J值: -9.64
D2日期: 2025-03-06 00:00:00, D2_J值: 26.33
J值差值: 35.96

股票 sh.600160 的J值变化:
D1日期: 2025-03-20 00:00:00, D1_J值: -1.12
D2日期: 2025-03-25 00:00:00, D2_J值: 2.52
J值差值: 3.64

股票 sh.600161 的J值变化:
D1日期: 2025-02-13 00:00:00, D1_J值: -12.86
D2日期: 2025-02-14 00:00:00, D2_J值: 10.68
J值差值: 23.54

股票 sh.600406 的J值变化:
D1日期: 2025-04-22 00:00:00, D1_J值: -5.15
D2日期

KeyError: 'code'

## 分析说明

In [ ]:
"""
计算全面指标
"""

# 1. 获取股票数据
stock_data = df_stock_2025[df_stock_2025["code"] == "sh.601888"].copy()
stock_data["date"] = pd.to_datetime(stock_data["date"])
stock_data = stock_data.sort_values('date').reset_index(drop=True)  # 重置索引，确保索引连续

# 2. 将价格相关的列转换为浮点数
numeric_columns = ['open', 'high', 'low', 'close', 'preclose', 'volume', 'amount', 'turn']
for col in numeric_columns:
    stock_data[col] = pd.to_numeric(stock_data[col])

# 3. 计算MACD指标
macd, macd_signal, macd_hist = talib.MACD(
    stock_data['close'],
    fastperiod=12,
    slowperiod=26,
    signalperiod=9
)

# 4. 计算KDJ指标
def calculate_kdj(data, n=9, m1=3, m2=3):
    data = data.copy()
    
    # 计算RSV
    low_list = data['low'].rolling(window=n, min_periods=1).min()
    high_list = data['high'].rolling(window=n, min_periods=1).max()
    rsv = (data['close'] - low_list) / (high_list - low_list) * 100
    
    # 计算K、D、J值
    k = np.zeros(len(data))
    d = np.zeros(len(data))
    
    # 初始化第一个值
    k[0] = 50
    d[0] = 50
    
    # 计算其他值
    for i in range(1, len(data)):
        k[i] = (m1 - 1) * k[i-1] / m1 + rsv[i] / m1
        d[i] = (m2 - 1) * d[i-1] / m2 + k[i] / m2
    
    # 计算J值
    j = 3 * k - 2 * d
    
    return pd.Series(k), pd.Series(d), pd.Series(j)

# 计算KDJ
k, d, j = calculate_kdj(stock_data)

# 5. 计算BOLL指标
upper, middle, lower = talib.BBANDS(
    stock_data['close'],
    timeperiod=20,
    nbdevup=2,
    nbdevdn=2,
    matype=0
)

# 6. 将计算结果添加到数据框中
stock_data['MACD'] = macd
stock_data['MACD_signal'] = macd_signal
stock_data['MACD_hist'] = macd_hist
stock_data['KDJ_K'] = k
stock_data['KDJ_D'] = d
stock_data['KDJ_J'] = j
stock_data['BOLL_upper'] = upper
stock_data['BOLL_middle'] = middle
stock_data['BOLL_lower'] = lower

# 7. 获取2025/4/23的指标值
target_date = '2025-04-25'
result = stock_data[stock_data['date'] == target_date]

if len(result) == 0:
    print(f"未找到 {target_date} 的数据")
else:
    result = result.iloc[0]
    
    # 8. 打印结果
    print(f"\n=== {result['code']} ({target_date}) 技术指标分析 ===")
    print("\nMACD指标:")
    print(f"MACD值: {result['MACD']:.4f}")
    print(f"MACD信号线: {result['MACD_signal']:.4f}")
    print(f"MACD柱状图: {result['MACD_hist']:.4f}")

    print("\nKDJ指标:")
    print(f"K值: {result['KDJ_K']:.4f}")
    print(f"D值: {result['KDJ_D']:.4f}")
    print(f"J值: {result['KDJ_J']:.4f}")

    print("\nBOLL指标:")
    print(f"上轨: {result['BOLL_upper']:.4f}")
    print(f"中轨: {result['BOLL_middle']:.4f}")
    print(f"下轨: {result['BOLL_lower']:.4f}")

    # 9. 简单的技术分析
    print("\n=== 技术分析 ===")

    # MACD分析
    if result['MACD_hist'] > 0:
        macd_signal = "MACD处于上升趋势"
    else:
        macd_signal = "MACD处于下降趋势"
    print(f"MACD分析: {macd_signal}")

    # KDJ分析
    if result['KDJ_J'] > 100:
        kdj_signal = "KDJ超买"
    elif result['KDJ_J'] < 0:
        kdj_signal = "KDJ超卖"
    else:
        if result['KDJ_J'] > result['KDJ_K'] and result['KDJ_K'] > result['KDJ_D']:
            kdj_signal = "KDJ金叉，看多信号"
        elif result['KDJ_J'] < result['KDJ_K'] and result['KDJ_K'] < result['KDJ_D']:
            kdj_signal = "KDJ死叉，看空信号"
        else:
            kdj_signal = "KDJ处于盘整区间"
    print(f"KDJ分析: {kdj_signal}")

    # BOLL分析
    close = result['close']
    if close > result['BOLL_upper']:
        boll_signal = "股价突破布林上轨，超买区间"
    elif close < result['BOLL_lower']:
        boll_signal = "股价突破布林下轨，超卖区间"
    else:
        boll_signal = "股价在布林带中轨运行，趋势盘整"
    print(f"BOLL分析: {boll_signal}")

## 相关性分析

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from data_loader import DataLoader
from strategy import TradingStrategy
import matplotlib as mpl
import platform

# 设置matplotlib中文字体
if platform.system() == 'Windows':
    plt.rcParams['font.sans-serif'] = ['SimHei']  # Windows系统
elif platform.system() == 'Linux':
    plt.rcParams['font.sans-serif'] = ['DejaVu Sans']  # Linux系统
elif platform.system() == 'Darwin':
    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']  # macOS系统
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

# 设置seaborn样式
sns.set_style("whitegrid")
sns.set_context("paper", font_scale=1.5)

class MetricsAnalyzer:
    def __init__(self, signals_df):
        self.signals = signals_df
        
    def analyze_correlations(self):
        """分析各指标与收益率的相关性"""
        # 选择需要分析的指标
        metrics = ['D1_J值', 'D2_J值', 'D1_WR14', 'D2_WR14', 'D1_WR28', 'D2_WR28']
        
        # 计算相关性
        corr_data = self.signals[metrics + ['D1-D2收益率']]
        correlation = corr_data.corr()['D1-D2收益率'].sort_values()
        
        print("\n=== 指标与收益率的相关性分析 ===")
        print(correlation)
        
        # 绘制相关性热图
        plt.figure(figsize=(12, 10))
        sns.heatmap(corr_data.corr(), annot=True, cmap='coolwarm', center=0, fmt='.2f')
        plt.title('指标相关性热图', pad=20, fontsize=16)
        plt.tight_layout()
        plt.savefig('correlation_heatmap.png', dpi=300, bbox_inches='tight')
        plt.close()
        
    def analyze_by_wr_zones(self):
        """根据WR指标的不同区间分析收益率"""
        def categorize_wr(value):
            if value > -20:
                return '超买区间 (>-20)'
            elif value < -80:
                return '超卖区间 (<-80)'
            else:
                return '中性区间 (-80~-20)'
        
        # 对WR14和WR28进行分类
        self.signals['D1_WR14_Zone'] = self.signals['D1_WR14'].apply(categorize_wr)
        self.signals['D1_WR28_Zone'] = self.signals['D1_WR28'].apply(categorize_wr)
        
        # 分析WR14区间的收益率
        wr14_analysis = self.signals.groupby('D1_WR14_Zone').agg({
            'D1-D2收益率': ['count', 'mean', 'std', 'median'],
            '持仓天数': 'mean'
        })
        
        # 分析WR28区间的收益率
        wr28_analysis = self.signals.groupby('D1_WR28_Zone').agg({
            'D1-D2收益率': ['count', 'mean', 'std', 'median'],
            '持仓天数': 'mean'
        })
        
        print("\n=== WR14区间收益率分析 ===")
        print(wr14_analysis)
        print("\n=== WR28区间收益率分析 ===")
        print(wr28_analysis)
        
        # 绘制箱线图
        plt.figure(figsize=(15, 7))
        
        plt.subplot(1, 2, 1)
        sns.boxplot(x='D1_WR14_Zone', y='D1-D2收益率', data=self.signals)
        plt.title('WR14区间收益率分布', pad=20, fontsize=14)
        plt.xlabel('WR14区间', fontsize=12)
        plt.ylabel('收益率 (%)', fontsize=12)
        plt.xticks(rotation=45)
        
        plt.subplot(1, 2, 2)
        sns.boxplot(x='D1_WR28_Zone', y='D1-D2收益率', data=self.signals)
        plt.title('WR28区间收益率分布', pad=20, fontsize=14)
        plt.xlabel('WR28区间', fontsize=12)
        plt.ylabel('收益率 (%)', fontsize=12)
        plt.xticks(rotation=45)
        
        plt.tight_layout()
        plt.savefig('wr_returns_boxplot.png', dpi=300, bbox_inches='tight')
        plt.close()
        
    def analyze_by_j_value(self):
        """分析J值与收益率的关系"""
        # 将J值分组
        self.signals['D1_J值_Range'] = pd.qcut(self.signals['D1_J值'], q=5, labels=[
            '极低', '较低', '中等', '较高', '极高'
        ])
        
        # 分析不同J值区间的收益率
        j_analysis = self.signals.groupby('D1_J值_Range').agg({
            'D1-D2收益率': ['count', 'mean', 'std', 'median'],
            '持仓天数': 'mean',
            'D1_J值': ['min', 'max']
        })
        
        print("\n=== J值区间收益率分析 ===")
        print(j_analysis)
        
        # 绘制J值与收益率的散点图
        plt.figure(figsize=(12, 8))
        plt.scatter(self.signals['D1_J值'], self.signals['D1-D2收益率'], alpha=0.5)
        plt.xlabel('D1日J值', fontsize=12)
        plt.ylabel('D1-D2收益率 (%)', fontsize=12)
        plt.title('J值与收益率的关系', pad=20, fontsize=16)
        plt.grid(True)
        plt.savefig('j_value_returns_scatter.png', dpi=300, bbox_inches='tight')
        plt.close()
        
    def analyze_combined_signals(self):
        """分析J值和WR指标组合条件下的收益率"""
        def get_combined_signal(row):
            j_signal = "J值低" if row['D1_J值'] < -10 else "J值高"
            wr_signal = "WR超卖" if row['D1_WR14'] < -80 else ("WR超买" if row['D1_WR14'] > -20 else "WR中性")
            return f"{j_signal}_{wr_signal}"
        
        self.signals['Combined_Signal'] = self.signals.apply(get_combined_signal, axis=1)
        
        # 分析组合信号的收益率
        combined_analysis = self.signals.groupby('Combined_Signal').agg({
            'D1-D2收益率': ['count', 'mean', 'std', 'median'],
            '持仓天数': 'mean'
        })
        
        print("\n=== 组合信号收益率分析 ===")
        print(combined_analysis)
        
        # 绘制组合信号的箱线图
        plt.figure(figsize=(14, 8))
        sns.boxplot(x='Combined_Signal', y='D1-D2收益率', data=self.signals)
        plt.title('组合信号收益率分布', pad=20, fontsize=16)
        plt.xlabel('组合信号类型', fontsize=12)
        plt.ylabel('收益率 (%)', fontsize=12)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('combined_signals_boxplot.png', dpi=300, bbox_inches='tight')
        plt.close()

def main():
    # 加载数据
    stock_data_path = "/home/kennys/MineX/QuantTrading/dataset/沪深300-2024年至今数据.csv"
    hs300_constituents_path = "/home/kennys/MineX/QuantTrading/dataset/沪深300成分股.csv"
    data_loader = DataLoader(stock_data_path, hs300_constituents_path)
    
    stock_data = data_loader.load_stock_data()
    hs300_constituents = data_loader.load_hs300_constituents()
    
    stock_data = pd.merge(stock_data, hs300_constituents[['code', 'code_name']], on='code', how='inner')
    
    # 运行策略获取信号
    strategy = TradingStrategy()
    prepared_data = strategy.prepare_data(stock_data)
    signals = strategy.find_trading_signals(prepared_data)
    
    if len(signals) > 0:
        # 添加股票名称
        signals = pd.merge(signals, hs300_constituents[['code', 'code_name']], on='code', how='left')
        
        # 创建分析器并进行分析
        analyzer = MetricsAnalyzer(signals)
        
        # 运行各项分析
        analyzer.analyze_correlations()
        analyzer.analyze_by_wr_zones()
        analyzer.analyze_by_j_value()
        analyzer.analyze_combined_signals()
        
        print("\n分析结果已保存为图表文件：")
        print("1. correlation_heatmap.png - 指标相关性热图")
        print("2. wr_returns_boxplot.png - WR指标区间收益率分布")
        print("3. j_value_returns_scatter.png - J值与收益率散点图")
        print("4. combined_signals_boxplot.png - 组合信号收益率分布")
    else:
        print("\n未找到交易信号，无法进行分析")

if __name__ == "__main__":
    main() 